# Extend modelled climate record

The notebook extends the climate record for an existing Source model using Rainfall and PET grids for a new time period.

This notebook requires a few python packages that don't come with Anaconda:

* `veneer-py` for accessing Source via Veneer: [Install Instructions](https://github.com/flowmatters/veneer-py/blob/master/doc/training/0Setup_and_Hello_World.ipynb)
* A set of third party packages for spatial analysis: Installation instructions also at [https://github.com/flowmatters/veneer-py/blob/master/doc/training/0Setup_and_Hello_World.ipynb](https://github.com/flowmatters/veneer-py/blob/master/doc/training/0Setup_and_Hello_World.ipynb)
* `climate-utils` for computing zonal average time series [Install instructions](https://github.com/flowmatters/climate-utils)
* My slightly modified version of `python-rasterstats`. Run the following from the command prompt:

```
pip uninstall python-rasterstats
pip install https://github.com/joelrahman/python-rasterstats/archive/percent_cover.zip
```

* Probably also worth installing the `dsed-py` package while you're at it. [Instructions](https://github.com/flowmatters/dsed-py)

In [61]:
%matplotlib inline
import veneer
from veneer.actions import switch_data_source
import climate_utils
import shapely
from climate_utils import zonal
import rasterio
import os
import string
from glob import glob
import numpy as np
import pandas as pd
import sys
from datetime import datetime,timedelta
import requests
import json

## Settings

The following settings allow you to set machine and model specific options.

Relating to:

* Where the climate data is stored (`BASE_DIR`, `VARIABLES`, `PATTERNS`) - you probably only need to set `BASE_DIR`
* A bit about how things are configured in Source: `DATA_SOURCE`, `DATA_SOURCE_NAMING` and what you want things named after it has run `NEW_DATA_SOURCE`
* Dates for the current climate and the extension
* The projection for the Source model - in the example, `EPSG:3577` specifies Australian Albers
* The `PORT` that Veneer is running on

In [62]:
# Settings
# Information about the climate data on disk
BASE_DIR=os.path.join('D:'+os.path.sep,'Jupyter_notebooks','ExtendClimate_WT')
VARIABLES=['Rainfall','PET']
PATTERNS={
    'Rainfall':'${date}_rai.txt',
    'PET':'${date}_mwet.txt',
}

FOLDER_NAME={
    'Rainfall':'WT_RAIN_1960_2017',
    'PET':'WT_PET_1960_2017',

}

# Information about the model structure
DATA_SOURCE='Climate Data'                      # Where the climate data is stored within Source
EXISTING_DATA_SOURCE=DATA_SOURCE
NEW_DATA_SOURCE='Extended Climate Data'         # Where it will be stored after this notebook (needs to be different)

DATA_SOURCE_NAMING=string.Template('${variable} for ${catchment}') # How the columns are named

START_EXISTING=datetime(1960,1,1)
END_EXISTING=datetime(2015,11,30)

START_EXTENSION=END_EXISTING+timedelta(1)
END_EXTENSION=datetime(2017,6,30)

CRS='epsg:3577'   # Australian Albers
PORT=19876 # Where Veneer is running - will be 9876 if you only have one copy running

STORAGE_TO_CATCHMENT_MAPPING='storage_climate_sources.csv'

In [63]:
# Scaling factors for rainfall in particular catchments
SCALE_RAINFALL=True
SCALING_FACTORS = pd.read_csv('catchment_rainfall_scaling_factors.csv')

In [64]:
STORAGE_CLIMATE_SOURCES=pd.read_csv(STORAGE_TO_CATCHMENT_MAPPING)

In [65]:
v = veneer.Veneer(port=PORT)


## Get the catchment map.

We need the catchment map (from Source) and we need it projected to WGS84

In [66]:
network = v.network()
network = network.as_dataframe()
network.crs={'init':CRS}

*** /network ***


In [67]:
network_wgs84 = network.to_crs({'init':'epsg:4326'})
catchments_wgs84 = network_wgs84[network_wgs84['feature_type']=='catchment']
#catchments_wgs84 = catchments_wgs84['geometry','id']

In [68]:
#catchments_wgs84.drop?

In [69]:
catchments_wgs84

,areaInSquareMeters,feature_type,from_node,geometry,icon,link,name,schematic_location,to_node,id
1179,15764400,catchment,NaN,(POLYGON ((145.9327050891361 -17.8838583645656...,NaN,/network/link/183,SC #184,NaN,NaN,/network/catchments/1615
1180,59400,catchment,NaN,(POLYGON ((145.9357001424956 -17.9250007414258...,NaN,/network/link/184,SC #185,NaN,NaN,/network/catchments/1624
1181,46549800,catchment,NaN,(POLYGON ((145.9539235498933 -17.9896288206283...,NaN,/network/link/185,SC #186,NaN,NaN,/network/catchments/1633
1182,55780200,catchment,NaN,(POLYGON ((145.9421586526771 -18.0312173010038...,NaN,/network/link/186,SC #187,NaN,NaN,/network/catchments/1642
1183,48488400,catchment,NaN,(POLYGON ((145.5495582807318 -16.7010962408303...,NaN,/network/link/90,SC #91,NaN,NaN,/network/catchments/1651
1184,183717900,catchment,NaN,(POLYGON ((145.1281017053987 -17.5344036320863...,NaN,/network/link/314,SC #315,NaN,NaN,/network/catchments/1660
1185,253800,catchment,NaN,(POLYGON ((145.8019515214612 -18.1093481008826...,NaN,/network/link/190,SC #191,NaN,NaN,/network/catchments/1669
1186,23149800,catchment,NaN,(POLYGON ((145.908534313682 -17.92402886830967...,NaN,/network/link/182,SC #183,NaN,NaN,/network/catchments/1678
1187,47106900,catchment,NaN,(POLYGON ((145.4015529975092 -16.1392011634284...,NaN,/network/link/10,SC #11,NaN,NaN,/network/catchments/1687
1188,31871700,catchment,NaN,(POLYGON ((145.0808813139037 -16.1657435803095...,NaN,/network/link/30,SC #31,NaN,NaN,/network/catchments/1696


## Calculate the time series for the extension period

We loop over the variables (rainfall and pet), for each one we use the climate_utils package to compute the time series for each catchment

The following parameter specifies how the data is stored: (it says AWAP, but also works for SILO):

`zonal.awap_ascii_by_year(os.path.join(BASE_DIR,variable.lower()),PATTERNS[variable],'%Y%m%d'`


In [70]:
extensions={}
for variable in VARIABLES:
    print('Processing %s'%variable)
    extensions[variable] = zonal.compute_catchment_time_series(
        variable.lower(),
        catchments_wgs84,
        pd.date_range(START_EXTENSION,END_EXTENSION),
        zonal.awap_ascii_by_year(os.path.join(BASE_DIR,FOLDER_NAME[variable]),PATTERNS[variable],'%Y%m%d'),
        '${variable} for ${catchment}')


Processing Rainfall
. . 

C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)


. . . . . . 

C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3

. . . . . . 

...

C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3

. . . . . . 

C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3

. . . . . . 

C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3\lib\site-packages\climate_utils\zonal.py:57: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  data,transform = data_loader(variable,ts)
C:\Anaconda3

In [71]:
extensions['Rainfall']

,rainfall for SC #1,rainfall for SC #10,rainfall for SC #100,rainfall for SC #101,rainfall for SC #102,rainfall for SC #103,rainfall for SC #104,rainfall for SC #105,rainfall for SC #106,rainfall for SC #107,...,rainfall for SC #90517,rainfall for SC #91,rainfall for SC #92,rainfall for SC #93,rainfall for SC #94,rainfall for SC #95,rainfall for SC #96,rainfall for SC #97,rainfall for SC #98,rainfall for SC #99
2015-12-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-12-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-12-03,21.659519,48.327474,54.226863,61.112368,67.345438,63.450795,59.719627,55.583056,44.699285,88.777814,...,63.186380,18.926982,20.997557,24.732076,41.222641,36.851794,29.361306,85.682409,81.954092,50.286371
2015-12-04,2.533001,2.403538,0.000000,0.000000,0.074801,0.000118,0.009813,0.037771,0.056503,1.187061,...,0.086484,0.928924,0.622418,0.864676,0.000000,0.001756,0.209281,0.601675,2.663664,0.010991
2015-12-05,0.366500,0.543465,0.252749,0.600000,0.689528,1.224359,0.639251,0.606735,0.382072,0.039047,...,4.127033,0.654472,0.071671,0.000000,0.102773,0.000000,0.000000,0.100000,0.044303,0.140448
2015-12-06,0.000000,0.000000,0.133625,0.130050,0.282576,0.267945,0.090187,0.065688,0.071779,0.000000,...,0.100000,0.022213,0.040597,0.000000,0.002954,0.000000,0.000000,0.000000,0.000000,0.119439
2015-12-07,0.000000,0.000000,0.000000,0.000000,0.000054,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.027033,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-12-08,6.866501,7.328017,0.670240,0.950252,1.498172,0.838540,0.839251,0.868125,0.936095,3.650046,...,0.786484,1.169586,0.134433,0.196793,1.245425,0.996147,0.477792,2.823115,2.920428,0.543327
2015-12-09,1.899501,3.354870,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.099447,0.000000,0.000000,0.002954,0.000000,0.000000,0.000000,0.000000,0.000000
2015-12-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.016761,0.000000,0.000000,0.000000,0.000000,...,0.013516,0.487209,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Apply scaling factors

Here, we apply the scaling factors to the extension rainfall for particular catchments

SCALING_FACTORS    

In [72]:
SCALING_FACTORS

,SC,Scaling Factor,SCNumber
0,SC #9,1.29,9
1,SC #8,1.18,8
2,SC #6,1.47,6
3,SC #7,1.14,7
4,SC #42,1.45,42
5,SC #425,1.20,425
6,SC #486,1.27,486
7,SC #454,1.27,454
8,SC #438,1.04,438
9,SC #437,1.14,437


In [73]:
for (_,row) in SCALING_FACTORS.iterrows():
    col = 'rainfall for %s'%row.SC
    factor = row['Scaling Factor']
    print('Applying %d scaling factor to %s'%(factor,col))
    extensions['Rainfall'][col] *= factor

Applying 1 scaling factor to rainfall for SC #9
Applying 1 scaling factor to rainfall for SC #8
Applying 1 scaling factor to rainfall for SC #6
Applying 1 scaling factor to rainfall for SC #7
Applying 1 scaling factor to rainfall for SC #42
Applying 1 scaling factor to rainfall for SC #425
Applying 1 scaling factor to rainfall for SC #486
Applying 1 scaling factor to rainfall for SC #454
Applying 1 scaling factor to rainfall for SC #438
Applying 1 scaling factor to rainfall for SC #437
Applying 1 scaling factor to rainfall for SC #443
Applying 1 scaling factor to rainfall for SC #446
Applying 1 scaling factor to rainfall for SC #365
Applying 1 scaling factor to rainfall for SC #364
Applying 1 scaling factor to rainfall for SC #459
Applying 1 scaling factor to rainfall for SC #457


## Retrieve the existing climate data from Source...

(so that we can stitch the existing and the new climate data together)

**Note:** these steps can take quite a while

In [74]:
data_sources = v.data_sources()
df = data_sources.as_dataframe()
df[df.Name==EXISTING_DATA_SOURCE]

*** /dataSources ***


,FullName,Items,Name,Path,id
44,/Climate Data,"[{'DetailsAsCSV': None, 'UnitsForNewTS': None,...",Climate Data,/,/dataSources/Climate Data


In [75]:
veneer.general.PRINT_URLS=True
%time existing_climate_data_record = v.data_source(EXISTING_DATA_SOURCE)
existing_climate_data = existing_climate_data_record['Items'][0]['Details']

*** /dataSources/Climate Data ***
Wall time: 51.6 s


## Combine the climate sets

Now we join the existing and the new data

In [76]:
# Put Rainfall and PET extensions into one dataframe
complete_ext = extensions['Rainfall'].join(extensions['PET'])

In [77]:
# Check that we've got the same columns in both the existing and the new data
set(existing_climate_data.columns)-set(complete_ext.columns)

set()

In [78]:
len(complete_ext.columns),len(existing_climate_data.columns)

(1034, 1034)

In [79]:
# Join!
new_climate = pd.concat([existing_climate_data,complete_ext])

In [80]:
new_climate

,pet for SC #1,pet for SC #10,pet for SC #100,pet for SC #101,pet for SC #102,pet for SC #103,pet for SC #104,pet for SC #105,pet for SC #106,pet for SC #107,...,rainfall for SC #90517,rainfall for SC #91,rainfall for SC #92,rainfall for SC #93,rainfall for SC #94,rainfall for SC #95,rainfall for SC #96,rainfall for SC #97,rainfall for SC #98,rainfall for SC #99
1960-01-01,7.500000,7.260033,7.351431,7.469976,7.442327,7.439130,7.500000,7.409317,7.200019,7.423253,...,0.000000,4.588860,1.074076,3.324182,4.173881,5.974589,6.530580,0.024751,0.408072,0.000000
1960-01-02,6.933419,6.913103,7.333069,7.400000,7.399905,7.400000,7.400000,7.393115,7.343675,7.135076,...,1.479620,3.885979,7.901128,7.795114,3.341679,4.340331,4.834803,1.024751,1.826411,2.008866
1960-01-03,6.933419,6.859173,7.150383,7.169976,7.255788,7.351894,7.200000,7.173215,7.128163,7.153263,...,4.273249,2.360149,2.767302,2.161941,2.723898,4.049416,4.290684,3.982234,4.352809,10.126437
1960-01-04,7.333419,7.075136,7.550383,7.569976,7.542327,7.539020,7.590174,7.542385,7.528169,7.542933,...,0.100000,0.157965,0.000000,0.000032,0.184457,0.189130,0.054112,2.274252,2.054472,0.986620
1960-01-05,7.333419,7.065185,7.699858,7.669976,7.655184,7.699944,7.700000,7.700000,7.700000,7.646376,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1960-01-06,7.200000,7.073894,7.233466,7.200000,7.312241,7.200000,7.200000,7.203442,7.228163,7.259660,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013164,0.000000
1960-01-07,7.100000,6.815869,7.350383,7.369976,7.342327,7.347386,7.390174,7.342385,7.328163,7.332603,...,0.519746,2.288167,5.126602,2.573494,2.075866,1.873374,1.744487,1.399002,1.263865,0.404514
1960-01-08,6.800000,6.512070,7.099858,7.069976,7.055184,7.099944,7.100000,7.100000,7.100000,7.006036,...,6.232488,3.052073,5.696286,7.241827,5.776112,10.357736,10.295032,18.243514,16.176327,10.757012
1960-01-09,6.833419,6.176914,6.451148,6.509927,6.389537,6.371575,6.570522,6.427155,6.384482,6.363913,...,0.433122,0.000000,0.437973,0.735651,0.018007,0.140882,0.327319,0.775249,1.855510,2.108544
1960-01-10,5.000257,4.507265,2.999575,2.909927,2.752329,2.771907,3.000000,3.000000,3.000019,3.481983,...,55.258637,0.107009,3.764309,6.604393,4.466920,11.398459,11.170796,58.049696,59.620133,71.548039


## Push the combined (extended) climate data up to Source as a new Data Source



In [81]:
v.create_data_source(NEW_DATA_SOURCE,new_climate,units='mm/day')


(200, None)

In [83]:
switch_data_source(v,EXISTING_DATA_SOURCE,NEW_DATA_SOURCE)

{'Exception': None, 'Response': None, 'StandardError': '', 'StandardOut': ''}

## Apply to storage...

At this point, the new data is loaded in Source and applied to all the subcatchments, including scaling as required.

The data isn't hooked up to the storages though..

In [84]:
STORAGE_CLIMATE_SOURCES

,Storage,SC
0,Koombaloomba,457
1,Tinaroo Falls,464
2,Cooperloade,460


In [85]:
veneer.general.PRINT_SCRIPTS=False
for (_,row) in STORAGE_CLIMATE_SOURCES.iterrows():
    v.model.node.assign_time_series(
        'RainfallInMetresPerSecond',
        'rainfall for SC #%s'%row.SC,
        NEW_DATA_SOURCE,
        nodes=row.Storage)
    v.model.node.assign_time_series(
        'EvaporationInMetresPerSecond',
        'pet for SC #%s'%row.SC,
        NEW_DATA_SOURCE,
        nodes=row.Storage)
    

## DONE.

Ok, now the new data is loaded in Source (with any scaling factors applied), the data is attached to the relevant subcatchments AND storages.

The old data sources still exist in Source - at the moment you need to delete them manually...



In [ ]:
v.drop_all_runs()

*** /runs ***


In [ ]:
v.run_model(start='01/07/1986',end=END_EXTENSION)

In [ ]:
import sys

In [ ]:
sys.version

In [ ]:
!pip list